In [1]:
# RESOURCES

import os
from pathlib import Path
from bifacial_radiance import *   
import numpy as np
import pandas as pd
import time


In [2]:
# ASSIGN WORKING DIRECTORY

testfolder = 'C:\\Users\\h-hugo18h\\Desktop\\Tyyssija'
if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Simulation will be stored in %s" % testfolder)


Simulation will be stored in C:\Users\h-hugo18h\Desktop\Tyyssija


In [3]:
# Assign parameters for the whole canopy model

simulationname = 'Tyyssija_Testing'

# Location:
lat = 60.448399         # Naantali
lon = 21.968503         # Naantali


# Module Parameters
moduletype = 'SaloSolarTest'


numpanels = 1           # Naantali has 6 modules along the y direction (N-S, modules are facing the South).
x = 1057  
y = 1778 
xgap = 17 #+ 0.10     # Leaving 10 centimeters between modules on x direction
ygap = 17 #+ 0.0575   # Leaving 10 centimeters between modules on y direction
zgap = 0                # no gap to torquetube.
sensorsy = 6*numpanels  # this will give 6 sensors per module, 1 per cell

# Cells parameters
cellLevelModule = True  # !!!
numcellsx = 10
numcellsy = 6
xcell = 166
ycell = 166
xcellgap = 2
ycellgap = 2

cellLevelModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

# TorqueTube Parameters
axisofrotationTorqueTube = False  # this is False by default if there is no torquetbue parameters
torqueTube = False

# SceneDict Parameters
pitch =  50       # Separation between module arrays, not needed in Naantali's case
albedo = 0.2        # Ground albedo
hub_height = 4000    # m  
nMods = 3           # eight modules in x direction.
nRows = 1           # number of array rows

azimuth_ang = 191   # Facing South-West
tilt = 15       # Tilt angle proposed 15, reality 17, measured with the pyranometer. 


In [4]:
# Create the model

demo = RadianceObj(simulationname, path = testfolder)  
demo.setGround(albedo) 

epwfile = 'EPWs\FIN_Tampere.029440_IWEC2V.epw' # This EPW was modfied with Naantali irradiance and weather data
metdata = demo.readWeatherFile(epwfile, coerce_year = 2001)


# Making module with all the variables

module = demo.makeModule(name=moduletype, x=x, y=y, numpanels=numpanels, 
                           xgap=xgap, ygap=ygap, cellModule=cellLevelModuleParams)


# Create a scene with all the variables
sceneDict = {'tilt':tilt, 'pitch': 1, 'hub_height':hub_height, 'azimuth':azimuth_ang, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(module=moduletype, sceneDict=sceneDict)



path = C:\Users\h-hugo18h\Desktop\Tyyssija
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2001
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos

Module Name: SaloSolarTest
Module was shifted by 83.0 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.98 %
Module SaloSolarTest updated in module.json


In [15]:
# Adding objects to the scene, add as many as needed

scene = demo.makeScene(module=moduletype, sceneDict=sceneDict)

add_obj = 'objects/Building_01.rad'
demo.appendtoScene(radfile=scene.radfiles, customObject = add_obj, text="!xform -rz -11")

print('Objects succesfully loaded')


Objects succesfully loaded


In [24]:
start_time = pd.to_datetime('2001-08-25 12:0:0 +2')                          #Sample from 6 to 16, 12 hours total. Can be shorter, early and late hours do not match quite well
end_time = pd.to_datetime('2001-08-25 12:0:0 +2')
timeStamps = pd.date_range(start = start_time, end = end_time, freq ='1h')

for i in timeStamps:
    timestamp = metdata.datetime.index(pd.to_datetime(i))  # Make this timezone aware, use +2 for EET.
    demo.gendaylit(timestamp)
    
    files = demo.getfilelist()
    octfile = demo.makeOct(files)
    
    #!rvu -vf views\front_SE_ok.vp -e .01 -pe 0.4 Tyyssija_Testing.oct
    !objview materials/ground.rad SaloSolarTest_C_3869.81402_rtr_1.00000_tilt_15.00000_3modsx1rows_origin0,0.rad objects/Building_01.rad objects/Building_01.rad objects/Panel_Racks_without_Module_Faces_01.rad


Created Tyyssija_Testing.oct
	oconv -f materials/ground.rad SaloSolarTest_C_3869.81402_rtr_1.00000_tilt_15.00000_3modsx1rows_origin0,0.rad objects/Building_01.rad objects/Building_01.rad objects/Panel_Racks_without_Module_Faces_01.rad C:/Users/H-HUGO~1/AppData/Local/Temp/SFFB4qrH9m/lt17916.rad > C:/Users/H-HUGO~1/AppData/Local/Temp/SFFB4qrH9m/ov17916.oct
set ROPT=-dp 128 -ar 16 -ds 0 -dt .2 -dc .25 -dr 0 -ss 0 -st .5 -af C:/Users/H-HUGO~1/AppData/Local/Temp/SFFB4qrH9m/af17916.amb -aa .3 -ad 256 -as 0 -av 1 1 1 -lr 6 -lw .003
set VIEW=-vtv -vp 2.49e+05 2.18e+05 1.74e+05 -vd -9.29e+04 -9.29e+04 -9.29e+04 -vu 0 0 1 -vh 45 -vv 45
	rvu $VIEW $ROPT -ps 8 -pt .16 -R C:\Users\H-HUGO~1\AppData\Local\Temp\SFFB4qrH9m/ov17916.rif -o qt -pe .5 C:/Users/H-HUGO~1/AppData/Local/Temp/SFFB4qrH9m/ov17916.oct


QWindowsNativeFileDialogBase::onSelectionChange () 0
QWindowsNativeFileDialogBase::onSelectionChange (QUrl("file:///C:/Users/h-hugo18h/Desktop/Tyyssija/views")) 1
QWindowsNativeFileDialogBase::onSelectionChange () 0
QWindowsNativeFileDialogBase::onSelectionChange (QUrl("file:///C:/Users/h-hugo18h/Desktop/Tyyssija/views/close_03.vp")) 1
QWindowsNativeFileDialogBase::onSelectionChange (QUrl("file:///C:/Users/h-hugo18h/Desktop/Tyyssija/views/close_03.vp")) 1
QWindowsNativeFileDialogBase::onSelectionChange () 0
QWindowsNativeFileDialogBase::onSelectionChange (QUrl("file:///C:/Users/h-hugo18h/Desktop/Tyyssija/views/close.vp")) 1


In [7]:
# Main Loop - Sampling across 5 sensors


# Define time stamps
 
start_time = pd.to_datetime('2001-08-25 7:0:0 +2')                          #Sample from 6 to 16, 12 hours total. Can be shorter, early and late hours do not match quite well
end_time = pd.to_datetime('2001-08-25 18:0:0 +2')
timeStamps = pd.date_range(start = start_time, end = end_time, freq ='1h')

# Data frame to store results
A = pd.DataFrame([[0.0,0.0,0.0,0.0,0.0]], index=['2001-08-25 5:0:0 +2'], columns=['sen1', 'sen2', 'sen3', 'sen4', 'sen5'])



# Define number of sensors and sampling position

file_corrected = 'objects\\Solitek-Bifacial350W_C_3.35809_rtr_1.00000_tilt_17.09500_8modsx1rows_origin0,0C.rad'
sensorsy = 1
refCells = 5
coordinates = [-6.61765, 3.58459, 5.33875], [-0.0327869, -0.169006, 4.59204], [0.429647, 2.21101, 5.33763], [0.942082, 4.85493, 6.16581], [7.47631, 0.841604, 5.33773]

cellsResult =  []


for i in timeStamps:
    print('\nSampling Irradiance at ', i)
    timestamp = metdata.datetime.index(pd.to_datetime(i))  # Make this timezone aware, use +2 for EET.
    demo.gendaylit(timestamp)
    
    files = demo.getfilelist()
    files[2] = file_corrected
    print('\nFile list ', files, '\n')
    octfile = demo.makeOct(files)
    
    #!rvu -vf views\front_SE_ok.vp -e .01 -pe 0.4 Naantali02_wwater_pwood.oct
    
    analysis = AnalysisObj(octfile, demo.name)
    frontscan, backscan = analysis.moduleAnalysis(scene, sensorsy = sensorsy)

    for cell in range(refCells):
    # Defining starting point for Frontscan and Backscan
    
        try:
            frontscan['xstart'] =  coordinates[cell][0] 
            frontscan['ystart'] =  coordinates[cell][1]
            frontscan['zstart'] =  coordinates[cell][2]

            backscan['xstart'] = frontscan['xstart']
            backscan['ystart'] = frontscan['ystart']
            backscan['zstart'] = frontscan['zstart'] - 0.04
            analysis.analysis(octfile, simulationname + "Sensor_%s"%(cell+1), frontscan, backscan)
            cellsResult.append(round(analysis.Wm2Back[0], 2))
        except:
            pass
    print(cellsResult)
    B = pd.DataFrame([cellsResult], index=[i], columns=['sen1', 'sen2', 'sen3', 'sen4', 'sen5'])
    A = pd.concat([A,B])
    cellsResult =  []

A.to_csv('sensors_Sim%s_allstructures.csv'%('Aug25_10'), sep=';', decimal = ',') 
print(A)    
print('\nDone.\n')



Sampling Irradiance at  2001-08-25 07:00:00+02:00

File list  ['materials\\ground.rad', 'skies\\sky2_60.44_21.96_2001-08-25_0700.rad', 'objects\\Solitek-Bifacial350W_C_3.35809_rtr_1.00000_tilt_17.09500_8modsx1rows_origin0,0C.rad'] 

Created Naantali02_Sensors.oct
Linescan in process: Naantali02_SensorsSensor_1_Front
Linescan in process: Naantali02_SensorsSensor_1_Back
Saved: results\irr_Naantali02_SensorsSensor_1.csv
Linescan in process: Naantali02_SensorsSensor_2_Front
Linescan in process: Naantali02_SensorsSensor_2_Back
Saved: results\irr_Naantali02_SensorsSensor_2.csv
Linescan in process: Naantali02_SensorsSensor_3_Front
Linescan in process: Naantali02_SensorsSensor_3_Back
Saved: results\irr_Naantali02_SensorsSensor_3.csv
Linescan in process: Naantali02_SensorsSensor_4_Front
Linescan in process: Naantali02_SensorsSensor_4_Back
Saved: results\irr_Naantali02_SensorsSensor_4.csv
Linescan in process: Naantali02_SensorsSensor_5_Front
Linescan in process: Naantali02_SensorsSensor_5_Back


In [11]:
# Generate the Radiance model view

octfile = demo.makeOct(files)
# Activate any of the views available at views\
#!rvu -vf views\closeup_module.vp -e .01 -pe 0.4 Naantali02_Sensors.oct
#!rvu -vf views\front.vp -e .01 -pe 0.4 Naantali02_Sensors.oct
#!rvu -vf views\sensor1.vp -e .01 -pe 0.4 Naantali02_Sensors.oct
#!rvu -vf views\top_ok.vp -e .01 -pe 0.4 Naantali02_Sensors.oct
#rvu -vf views\topcorner_OK.vp -e .01 -pe 0.4 Naantali02_Sensors.oct
!rvu -vf views\front_SE_ok.vp -e .01 -pe 0.4 Naantali02_Sensors.oct

Created Tyyssija_Testing.oct


C:\Users\h-hugo18h\Anaconda3\lib\site-packages\bifacial_radiance\main.py:1901: Warning: oconv: system - cannot open scene file "oconv": No such file or directory
  warnings.warn(err[9:], Warning)
